In [9]:
#import the required Python packages

import pandas as pd
#import datetime   #check this
import pyeto
import numpy as np
import math
from pandas import DataFrame

from pyeto import fao
#from datetime import datetime

#import dateutil     #dateutil module provides powerful extensions to the standard datetime module
#from dateutil import parser  #This module reads the given date in string and convert it to date format or timestamps,it represent a date and/or time from most known formats 

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

In [10]:
#df=pd.read_csv('pilot_output2_clean.csv') #This is a sample dataset with 100 points only. 
#df=pd.read_excel('nwsas_gis_output_20190607_clean.xlsx')
df=pd.read_csv('nwsas_gis_output_20190607_clean.csv')

In [11]:
######### input variables ##############3
pumping_hours_per_day=10 #is this an assumption?? 
deff= 1
aeff= 0.45  #Surface irrigation, low efficiency:0.45 , high: 0.65  // Drip irrigation: 0.9
ky_dict = {'dates':0.5,'wheat':0.87} #ky values for each crop
kc_dict = {'dates': [0.56,0.7,0.7,0.56], 'wheat':[0.7,1.15,1.15,0.25]} #kc values for each crop in each growing season

#defines the agricultural mode for each region, i.e. specifying the percentage of land cultivated per crop type 
df['Mode']=[{'dates':0.8,'wheat':0.2}]*df.shape[0]
#df.loc[df['NAME_1']=='Beni Walid','Mode'] = [{'dates':0,'wheat':1}]

output_file = 'sur45_waterdemand_20190611' #output file name

In [12]:
#Estimating the DAILY ETo function:
#shf – Soil heat flux (G) [MJ m-2 day-1] (default is 0.0, which is reasonable for a daily or 10-day time steps).

def evap_i(lat,elev,wind,srad,tmin,tmax,tavg,month):
    if month ==1:
        J = 15
    else:
        J = 15 + (month-1)*30
        
    latitude = pyeto.deg2rad(lat)
    atmosphericVapourPressure = pyeto.avp_from_tmin(tmin)
    saturationVapourPressure = pyeto.svp_from_t(tavg)
    ird = pyeto.inv_rel_dist_earth_sun(J)
    solarDeclination = pyeto.sol_dec(J)
    sha = [pyeto.sunset_hour_angle(l, solarDeclination) for l in latitude]
    extraterrestrialRad = [pyeto.et_rad(x, solarDeclination,y,ird) for x, y in zip(latitude,sha)]
    clearSkyRad = pyeto.cs_rad(elev,extraterrestrialRad)
    netInSolRadnet = pyeto.net_in_sol_rad(srad*0.001, albedo=0.23)
    netOutSolRadnet = pyeto.net_out_lw_rad(tmin, tmax, srad*0.001, clearSkyRad, atmosphericVapourPressure)
    netRadiation = pyeto.net_rad(netInSolRadnet,netOutSolRadnet)
    tempKelvin = pyeto.celsius2kelvin(tavg)
    windSpeed2m = wind
    slopeSvp = pyeto.delta_svp(tavg)
    atmPressure = pyeto.atm_pressure(elev)
    psyConstant = pyeto.psy_const(atmPressure)
    
    return pyeto.fao56_penman_monteith(netRadiation, tempKelvin, windSpeed2m, saturationVapourPressure, atmosphericVapourPressure, slopeSvp, psyConstant, shf=0.0)

In [13]:
for i in range(1,13):
    df['ETo_{}'.format(i)]=0  ##To make sure the it is reset to zero

In [15]:
#calculate ETo for each row for each month 
## range(1,13) and .format(i): to generate monthly calculation of ETo
## df.iterrows() and use of .iloc[index]: To make sure the calculation will be repearted for each index point. 

for i in range(1,13):
    df['ETo_{}'.format(i)] = evap_i(df['lat'],df['elevation'],df['wind{}'.format(i)],df['srad{}'.format(i)],df['tmin{}'.format(i)],df['tmax{}'.format(i)],df['tavg{}'.format(i)],i)

KeyError: 'lat'

In [24]:
#Effective rainfall function 

def eff_rainfall(prec,eto):
    return (1.253*((prec**0.824)-2.935))*10**(0.001*eto)  #Find the source

In [25]:
#calculate eff rainfall for each row for each month
#This source: http://www.fao.org/docrep/S2022E/s2022e08.htm was initially used but the updated equaiton of the effective rainfall comes form another source: 

for i in range(1,13):
    df['eff_{}'.format(i)]=0
    
for i in range(1,13):
    df.loc[df['prec{}'.format(i)] < 12.5, 'eff_{}'.format(i)] = df['prec{}'.format(i)]/30
    df.loc[df['prec{}'.format(i)] >= 12.5, 'eff_{}'.format(i)] = eff_rainfall(df['prec{}'.format(i)],df['ETo_{}'.format(i)])/30 

In [26]:
#calculate kc based on the growing stage (month - planting, growing, harvesting season/month)
#introduce the kc function and its attributes

#def kc(plantation,Li,Ld,Lm,Le,kci,kcd,kcm,kce,isodate):  #initial code

def kc(plantation,Li,Ld,Lm,Le,kci,kcd,kcm,kce,isodate): #new code: Li1, Li2, kci1 and kci2 
    """
    Each crop goes through four growing stages: initial - development - mid-season and end-season (check FAO-56 chapter 6 for more details)

    Inputs:
    Plantation = plantation datetime 
    Li = length of the initial stage (in days)
    Ld = length of the development stage (in days)
    Lm = length of the mid-season stage (in days)
    Le = length of the end-season stage (in days)

    kci = crop coefficient 'kc' at the initial stage. In this stage the ckc value is constant and equal to kci
    kcm = crop coefficient 'kc' at the mid-season stage.  In this stage the ckc value is constant and equal to kcm
    kce = crop coefficient 'kc' at the end-season stage. In this stege the ckc value varies linearly between kce and kcm. check equation 66 (page 132, FAO56). 
    isodate = current date (optional)

    Outputs: 
    * ckc : current crop coefficient, which is constant in the initial and mid-season stages and varies linearly in the development (increasing) and end-season (declining) stages. 

    Some Examples:
    Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-01-20")
        >>> 0.15
     
    Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-02-10")
        >>> 0.774
     
    Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-03-12")
        >>> 1.19
     
    Kc(plantation="2014-01-01",Li=25,Ld=25,Lm=30,Le=20,Kci=0.15,Kcm=1.19,Kce=0.35,isodate="2014-04-06")
        >>> 0.559
    """
    #step 1: 
    
    plantation = pd.to_datetime(plantation, format='%d/%m') #converting the plantation input info to data time
    isodate = pd.to_datetime(isodate , format='%d/%m')  #converting the current date input info to data time
    test = ((isodate-plantation).days+1)%365   #The difference in days between the current day and the plantation day.
    
    # Setting the plantation date and the current date (this is not used)
    Jc = test   
    Jp = 0
    J = (Jc - Jp)%365  # %365 means the remaing days of the year
    
    #Step 2: Calculating the day of the year when each crop stage ends placing the date in the number of days year betweem 0 (1/jan) and 365 (31/Jan)
    JLi = Jp + Li    #end of initial stage = plantation date + lenght of initial stage
#     JLi2 = JLi1 + Li2
    JLd = JLi + Ld   #end of development stage = end of initial stage + length of development stage
    JLm = JLd + Lm   #end of mid-season stage = end of development stage + length of mid-season stage
    JLe = JLm + Le   #end of end-season stage = end of mid-season stage + length of end-season stage

    #step 3: calculating ckc based on the end of each stage date

    if Jc > Jp and Jc < JLe:   #if the current date is greater than the plantation date and it is greater than the end of end-season stage
        if Jc <= JLi:    
            ckc = kci  #if the current date is before the end of initial stage then ckc = kci the coefficient of the initial stege
#         elif Jc > JLi1 and Jc <=JLi2: #New: to account for two init stages
#             ckc = kci2
        elif Jc > JLi and Jc <=JLd:  #if the current date is betweeen the end of the intial stege and the end of the development stage, then ckc is computed based on equation 66 (page 132.FAO56)
            ckc = kci + ((Jc-JLi)/Ld * (kcd-kci))
        elif Jc > JLd and Jc <= JLm: 
            ckc = kcm
        elif Jc > JLm and Jc <= JLe:
            ckc = kcm + ((Jc-JLm)/Le * (kce-kcm))       
    else:
        ckc = 0
    
    return ckc


In [27]:
#calculate kc based on the growing stage (month - planting, growing, harvesting season/month)
mode = pd.read_excel('NWSAS_DATES_CC2_201906.xlsx')

#Note: The code here is adjusted to avoid the end of year issue. In other cases, the init1 and init2 are one stage init:
#pay attention to all changes, you may need to change this if the crop calendar change 

#Planting season: Initial Stage 1 (plant = init1+ init2 )
init1_start = pd.to_datetime(mode['init_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
init1_end = pd.to_datetime(mode['init_end'], format='%d/%m')
mode['init_start_day'] = init1_start.dt.day
mode['init_end_day'] = init1_end.dt.day
mode['init_days'] = ((init1_end - init1_start).dt.days+1) % 365 #Calculating the length of the planting season
# Li1 = abs(init1_end - init1_start).dt.days

#Planting season: Initial Stage 2 (plant = init1+ init2 )
# init2_start = pd.to_datetime(mode['init2_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
# init2_end = pd.to_datetime(mode['init2_end'], format='%d/%m')
# mode['init2_start_month'] = init2_start.dt.month
# mode['init2_end_month'] = init2_end.dt.month
# mode['init2_days'] = abs(init2_end - init2_start).dt.days #Calculating the length of the planting season
# Li2 = abs(init2_end - init2_start).dt.days

#growing 1: Development Stage (grow = dev)
dev_start = pd.to_datetime(mode['dev_start'], format='%d/%m')
dev_end = pd.to_datetime(mode['dev_end'], format='%d/%m')
mode['dev_start_month'] = dev_start.dt.month
mode['dev_end_month'] = dev_end.dt.month
mode['dev_days'] = ((dev_end - dev_start).dt.days+1) % 365
# Ld = abs(dev_end - dev_start).dt.days

#growing 2: Mid stage ( add : mid)
mid_start = pd.to_datetime(mode['mid_start'], format='%d/%m')
mid_end = pd.to_datetime(mode['mid_end'], format='%d/%m')
mode['mid_start_month'] = mid_start.dt.month
mode['mid_end_month'] = mid_end.dt.month
mode['mid_days'] = ((mid_end - mid_start).dt.days+1) % 365
# Lm = abs(mid_end - mid_start).dt.days

#Harvesting: Late stage (harv = late)
late_start = pd.to_datetime(mode['late_start'], format='%d/%m') #defining the plant start date from excil and setting the correct month and days sequence to read.
late_end = pd.to_datetime(mode['late_end'], format='%d/%m')
mode['late_start_day'] = late_start.dt.day
mode['late_end_day'] = late_end.dt.day
mode['late_days'] = ((late_end - late_start).dt.days+1) % 365 #Calculating the length of the planting season
# Le = abs(late_end - late_start).dt.days

In [28]:
for i in range(1,13):
    mode['kc_{}'.format(i)]=0
    
for index,row in mode.iterrows():
    crop = row['Mode']
    for i in range(0,12):
        init1_start = pd.to_datetime(mode['init_start'].iloc[index], format='%d/%m') #read the plant start date from excel. 
        day_start= (init1_start.day+1-31)%31   #what does this represent??   
        
        if (init1_start.day-1==30):
            month_start = (init1_start.month+1-12)%12  #next month
        else:
            month_start = (init1_start.month-12)%12  #the current month
            
        month_start = (month_start+i)%12
        if (month_start==0):
            month_start = 12
        mode.loc[index,'kc_{}'.format(month_start)] = kc(mode['init_start'].iloc[index],mode['init_days'].iloc[index],
                                                         mode['dev_days'].iloc[index],mode['mid_days'].iloc[index],
                                                         mode['late_days'].iloc[index],
                                                         kc_dict[crop][0],kc_dict[crop][1],kc_dict[crop][2],kc_dict[crop][3],
                                                         '{}/{}'.format(day_start,month_start))
        #print (kc)
        
        # reacall that def kc(plantation,Li,Ld,Lm,Le,kci,kcd,kcm,kce,isodate): 
        #Assuming that :
        #Li = plant_days
        #Ld = dev_days
        #lm = mid_days.
        #le = late_days
        #kci = 0.8 tabulated values FAO 
        #kcd = 0.9 tablated values FAO 
        #kcm = 1 tablated values FAO 
        #kce = 0.8 tabulated values FAO 
        #isodate = '{}/{}'.format(day_start,month_start) 
        


In [29]:

def get_harvest_fraction(i,crop,init,late):
    if i != 12:
        current_date = pd.to_datetime((i+1),format='%m')
    else:
        current_date = pd.to_datetime(1,format='%m')
    start = pd.to_datetime(mode.loc[mode['Mode']==crop,init+'_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
    length = mode.loc[mode['Mode']==crop,init+'_days'].iloc[0]
    days = ((current_date - start).iloc[0].days) % 365
    late_end = pd.to_datetime(mode.loc[mode['Mode']==crop,late+'_end'], format='%d/%m').iloc[0]
    all_days = ((late_end - start).iloc[0].days+1) % 365
    if all_days == 0:
        all_days = 365
   
    if days == 0:
        start = pd.to_datetime(mode.loc[mode['Mode']==crop,late+'_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
        length = mode.loc[mode['Mode']==crop,late+'_days'].iloc[0]
        days = ((current_date - start).iloc[0].days) % 365
        if days <= length:
            return 1 #- days / length
        else:
            return 0
    elif days <= length:
        return days / length
    elif days <= all_days:
        return 1
    else:
        return 0

In [30]:
for i in range (1,13):
    df['PCWR_{}'.format(i)]=0       #PCWR: Peak Crop Water Requirement (l/s/ha) or "Duty", Previously PDWR
    df['PWD_{}'.format(i)]=0        #PWD: Peak Water Demand in (l/s)
    df['SSWD_{}'.format(i)]=0       #SSWD: Seasonal Scheme Water Demand in (m3)
    
#STEP 1: Compute the ACWR from ETc - check FAO1992- page 43-

#acwr=row['ETo_{}'.format(i)]*30*row['kc_{}'.format(i)] - row['eff_{}'.format(i)]*30 - row['awc']/12 ))
#once the available water content layer is obtained, the last past should be added to the equation

for crop in mode['Mode']:
    for i in range(1,13):
        eto = f'ETo_{i}'
        kc = f'kc_{i}'
        eff = f'eff_{i}'
        acwr = f'ACWR_{i}_'+crop
        pcwr = f'PCWR_{i}'
        pwd = f'PWD_{i}'
        sswd = f'SSWD_{i}'
        df[kc+'_'+crop] = float(mode.loc[mode['Mode']==crop,kc])
        ky=ky_dict[crop] #Yield response factor coeff = 0.8 for date palms, source TABLE 53-FAO: http://www.fao.org/3/y4360e/y4360e0b.htm 
        df[acwr] = (df[eto]*30*df[kc+'_'+crop]*ky - df[eff]*30 - (0.12*df[eff])*30) #Assumption: awc=12% effective rainfall
        df.loc[df[acwr]<0,acwr] = 0
        df[pcwr] += ((df[acwr]*10)/30)*2*0.012
        
        df[f'harvest_{i}_'+crop] = df['area_ha'] * np.array([x[crop] for x in df['Mode']]) * get_harvest_fraction(i,crop,'init','late')
        df[pwd] += (df[pcwr] *(df[f'harvest_{i}_'+crop]*24))/(pumping_hours_per_day*aeff*deff)
        df[sswd] += (df[acwr]*10*(df[f'harvest_{i}_'+crop])/(aeff*deff))

In [31]:
# Calculating the annual precipitation: which is the sum of precipitation values

df['precipitation_annual']=df.filter(like='prec').sum(axis=1)  #Filter is used to specify the column of interest


In [32]:
final=df.groupby('NAME_1').sum()
summary = pd.DataFrame({'Irrigated area (ha)':final['area_ha'],
                        'Water intensity (m3/ha)':final.filter(like='SSWD_').sum(axis=1)/final['area_ha'],
                        'Water demand (Mm3)':final.filter(like='SSWD_').sum(axis=1)/1000000})
summary.round(decimals=3)
summary.T

NAME_1,Adrar,Beni Walid,Biskra,Djelfa,El Oued,Gabes,Ghardaia,Illizi,Jufrah,Kebili,Khenchela,Laghouat,Musrata,Nalut,Ouargla,Tamanrasset,Tataouine,Tebessa,Tozeur
Irrigated area (ha),24101.816848,8557.616421,5749.995666,3449.972096,68999.985344,2874.983928,34499.998422,1770.396215,7963.713904,27412.511949,1149.970213,5749.994721,21082.919486,5353.213079,32699.346634,4868.857546,1833.775565,4599.970966,9617.446033
Water intensity (m3/ha),19245.036317,18868.686141,12392.888756,12659.454676,14154.960584,12025.650107,15679.265990,17183.912109,15678.350448,13697.306515,13176.143353,12348.479518,10968.025543,12815.754144,15328.011836,18914.685240,12315.149799,13320.320933,13210.103206
Water demand (Mm3),463.840341,161.470978,71.259057,43.674765,976.692073,34.573551,540.934652,30.422333,124.857897,375.477579,15.152172,71.003692,231.237999,68.605463,501.215972,92.092908,22.583221,61.273090,127.047455


In [33]:
df.to_csv(output_file+'.csv',index=False)

In [34]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter(output_file + '.xlsx', engine='xlsxwriter')

## Convert the dataframe to an XlsxWriter Excel object.
#df.to_excel(writer, sheet_name='Test')
#summary.to_excel(writer, sheet_name='summary')

## Close the Pandas Excel writer and output the Excel file.
#writer.save()